In [65]:
# import numpy as np
import theano.tensor as T
import theano.tensor.nnet as F
import theano


class HiddenLayer(object):
    def __init__(self, rng, ins, n_in, n_out, w=None, b=None, f=T.tanh):
        # initial weight and bias
        if w is not isinstance(w, np.ndarray):
            w = numpy.asarray(
                    rng.uniform(
                        low=-numpy.sqrt(6. / (n_in + n_out)),
                        high=numpy.sqrt(6. / (n_in + n_out)),
                        size=(n_in, n_out)
                    )
                )
            if f == theano.tensor.nnet.sigmoid:
                w *= 4
        if b is not isinstance(b, np.ndarray):
            b = numpy.zeros((n_out,))
       
        w = theano.shared(w, borrow=True)
        b = theano.shared(b, borrow=True)
        
        self.outs = T.dot(ins, w) - b
        if f is not None:
            # not a linear layer
            self.outs = f(self.outs)
        self.params = [w, b]
        

class NNet(object):
    def __init__(self, data, gstep=0.01, epochs=1000, rng=None):
        self.data = data
        self.ins = data
        self.n_sample, self.n_in = data.shape
        self.gstep = gstep   # gradient step
        self.layers = []
        self.params = []
        self.epochs = epochs
        if rng is None:
            self.rng = np.random
        else:
            self.rng = rng
        
    def add_layer(self, n_out, w=None, b=None, f=T.tanh):
        layer = HiddenLayer(rng=self.rng,
                            ins=self.ins,
                            n_in=self.n_in,
                            n_out=n_out,
                            w=w, b=b, f=f)
        self.layers.append(layer)
        # the output of this layer is the input of next layer
        self.ins = layer.outs
        self.n_in = n_out
        self.params += layer.params
        
    def train(self):
        x = T.dmatrix("x")
        # calculate the gradients
        gparams = T.grad(cost, [param for param in self.params])
        updates = [(param, param - self.gstep * gparam) for param, gparam in zip(self.params, gparams)]
        _train = theano.function(
                  inputs=[x],
                  outputs=self.cost,
                  updates=updates,
                  on_unused_input='ignore')
        for i in range(self.epochs):
            err = _train(self.data)
            print("Epoch: %d; Distance: %f" %(i, err))
        
    
    def set_cost(self, cost):
        self.cost = cost
    
    def get_final_outs(self):
        return self.layers[-1].outs
    

N = 5 # training sample size
IN_FEATS = 500 # input feature space
EPOCHS = 1000 # train iteration

rng = np.random
D = (rng.randn(N, IN_FEATS))      # inputs

nnet = NNet(D)
# encoder
nnet.add_layer(100)
# decoder
nnet.add_layer(IN_FEATS)
cost = ((nnet.get_final_outs() - D) ** 2).sum()
nnet.set_cost(cost)
nnet.train()       

UnusedInputError: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 0 is not part of the computational graph needed to compute the outputs: x.
To make this error into a warning, you can pass the parameter on_unused_input='warn' to theano.function. To disable it completely, use on_unused_input='ignore'.

In [57]:
# Autoencoder
import numpy as np
import theano.tensor as T
import theano.tensor.nnet as F
import theano

N = 5 # training sample size
IN_FEATS = 50 # input feature space, for encoder
OUT_FEATS = 100 # output feature space, for decoder
EPOCHS = 1000

rng = np.random
D = (rng.randn(N, IN_FEATS))      # inputs

x = T.dmatrix("x")

e_w1 = theano.shared(rng.randn(IN_FEATS, OUT_FEATS), name="encode_w1")  # weights
e_b1 = theano.shared(np.zeros(OUT_FEATS), name="encode_b1")             # bias
d_w1 = theano.shared(rng.randn(OUT_FEATS, IN_FEATS), name="decode_w1")
d_b1 = theano.shared(np.zeros(IN_FEATS), name="decode_b1")

encoder = F.sigmoid(T.dot(x, e_w1) - e_b1)
decoder = F.sigmoid(T.dot(encoder, d_w1) - d_b1)
cost = ((decoder - x) ** 2).sum()
e_gw1, e_gb1, d_gw1, d_gb1 = T.grad(cost, [e_w1, e_b1, d_w1, d_b1])

train = theano.function(
          inputs=[x],
          outputs=[encoder, cost],
          updates=((e_w1, e_w1 - 0.01*e_gw1), (e_b1, e_b1 - 0.01*e_gb1),
                   (d_w1, d_w1 - 0.01*d_gw1), (d_b1, d_b1 - 0.01*d_gb1)))
predict = theano.function(inputs=[x], outputs=encoder)

for i in range(EPOCHS):
    pred, err = train(D)
    print("Epoch: %d; Distance: %f" %(i, err))
    

Epoch: 0; Distance: 373.895791
Epoch: 1; Distance: 360.755747
Epoch: 2; Distance: 348.399575
Epoch: 3; Distance: 338.452834
Epoch: 4; Distance: 330.290144
Epoch: 5; Distance: 322.663898
Epoch: 6; Distance: 316.602942
Epoch: 7; Distance: 312.397063
Epoch: 8; Distance: 309.162198
Epoch: 9; Distance: 306.440610
Epoch: 10; Distance: 303.915050
Epoch: 11; Distance: 301.306587
Epoch: 12; Distance: 298.484276
Epoch: 13; Distance: 295.499803
Epoch: 14; Distance: 292.609960
Epoch: 15; Distance: 290.045539
Epoch: 16; Distance: 287.727999
Epoch: 17; Distance: 285.489984
Epoch: 18; Distance: 283.387156
Epoch: 19; Distance: 281.625815
Epoch: 20; Distance: 280.146320
Epoch: 21; Distance: 278.788027
Epoch: 22; Distance: 277.490845
Epoch: 23; Distance: 276.247998
Epoch: 24; Distance: 275.062662
Epoch: 25; Distance: 273.932894
Epoch: 26; Distance: 272.856797
Epoch: 27; Distance: 271.848309
Epoch: 28; Distance: 270.925029
Epoch: 29; Distance: 270.097966
Epoch: 30; Distance: 269.362613
Epoch: 31; Distanc